In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time

In [2]:
sampleSize=100000
sample_rate=16000
quantization_channels=256
dilations=[2**i for i in range(9)]*5
skipDim=512
residualDim=32
filterSize=3

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
#device = 'cpu'
torch.set_default_tensor_type('torch.cuda.FloatTensor')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [5]:
def mu_law_encode(audio, quantization_channels=quantization_channels):
    '''Quantizes waveform amplitudes.'''
    mu = (quantization_channels - 1)*1.0
    # Perform mu-law companding transformation (ITU-T, 1988).
    # Minimum operation is here to deal with rare large amplitudes caused
    # by resampling.
    safe_audio_abs = np.minimum(np.abs(audio), 1.0)
    magnitude = np.log1p(mu * safe_audio_abs) / np.log1p(mu)
    signal = np.sign(audio) * magnitude
    # Quantize signal to the specified number of levels.
    #if(forX):return signal
    return ((signal + 1) / 2 * mu + 0.5).astype(int)
def mu_law_decode(output, quantization_channels=quantization_channels):
    '''Recovers waveform from quantized values.'''
    mu = quantization_channels - 1
    # Map values back to [-1, 1].
    signal = 2 * ((output*1.0) / mu) - 1
    # Perform inverse of mu-law transformation.
    magnitude = (1 / mu) * ((1 + mu)**np.abs(signal) - 1)
    return np.sign(signal) * magnitude

In [6]:
def readAudio(name):
    audio0, samplerate = sf.read(name, dtype='float32')
    return librosa.resample(audio0.T, samplerate, sample_rate).reshape(-1)
p=['./vsCorpus/origin_mix.wav','./vsCorpus/origin_mix.wav',
   './vsCorpus/origin_mix.wav','./vsCorpus/origin_mix.wav','./vsCorpus/pred_mix.wav']
xtrain,ytrain,xval,yval,xtest=readAudio(p[0]),readAudio(p[1]),readAudio(p[2]),readAudio(p[3]),readAudio(p[4])
xtrain,ytrain,xval,yval=xtrain[:-sampleSize],ytrain[:-sampleSize],xval[-sampleSize:],yval[-sampleSize:]
xtrain,xval,xtest=xtrain.reshape(1,1,-1),xval.reshape(1,1,-1),xtest.reshape(1,1,-1)
ytrain,yval=ytrain.reshape(1,-1),yval.reshape(1,-1)

In [7]:
xtrain=(xtrain-xtrain.mean())/xtrain.std()
xval=(xval-xtrain.mean())/xtrain.std()
xtest=(xtest-xtrain.mean())/xtrain.std()
ytrain,yval=mu_law_encode(ytrain),mu_law_encode(yval)

In [8]:
xtrain,ytrain,xval,yval,xtest = torch.from_numpy(xtrain).type(torch.float32),\
                                torch.from_numpy(ytrain).type(torch.LongTensor),\
                                torch.from_numpy(xval).type(torch.float32),\
                                torch.from_numpy(yval).type(torch.LongTensor),\
                                torch.from_numpy(xtest).type(torch.float32)

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        sd,qd,rd = skipDim,quantization_channels,residualDim
        self.causal = nn.Conv1d(in_channels=1,out_channels=rd,kernel_size=3,padding=1)
        self.layer=dict()
        for i, d in enumerate(dilations):
            self.layer['tanh'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=d,dilation=d)
            self.layer['sigmoid'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=d,dilation=d)
            self.layer['skip'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=sd,kernel_size=1,padding=0)
            self.layer['dense'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=1,padding=0)
        self.post1 = nn.Conv1d(in_channels=sd,out_channels=sd,kernel_size=1,padding=0)
        self.post2 = nn.Conv1d(in_channels=sd,out_channels=qd,kernel_size=1,padding=0)
        self.tanh,self.sigmoid = nn.Tanh(),nn.Sigmoid()

    def forward(self, x):
        x = self.causal(x)
        #print('x.shape',x.shape)
        skip_connections = torch.zeros([1,skipDim,x.shape[2]], dtype=torch.float32)
        #skip_connections = []
        #print('skip_connections',skip_connections.shape)
        for i, dilation in enumerate(dilations):
            xinput = x.clone()
            x1 = self.tanh(self.layer['tanh'+str(i)](x))
            #print('tanh.shape',x1.shape)
            x2 = self.sigmoid(self.layer['sigmoid'+str(i)](x))
            #print('sigmoid.shape',x2.shape)
            x = x1*x2
            #print('multi',x3.shape)
            skip_connections += self.layer['skip'+str(i)](x)
            #print(skip_connections.cpu()[:,:,:10])
            #cur =self.layer['skip'+str(i)](x3)
            #skip_connections.append(cur)
            #print('skip.shape',self.layer['skip'+str(i)](x).shape)
            x = self.layer['dense'+str(i)](x)
            #print('dense.shape',x.shape)
            x += xinput
        #skip_connections = torch.cat(skip_connections,dim=0)
        #skip_connections = torch.sum(skip_connections,dim=0,keepdim=True)
        x = self.post2(F.relu(self.post1(F.relu(skip_connections))))
        return F.log_softmax(x,dim=1)

model = Net().to(device)

#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = optim.Adam(model.parameters())#,weight_decay=1e-5)

In [15]:
#torch.sum(c,dim=0,keepdim=True)

In [16]:
#model = torch.load('torchmodel0.681600')

In [20]:
def val():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        data, target = xval.to(device), yval.to(device)
        output = model(data)
        val_loss = F.nll_loss(output, target).item()
    print('\nval set:  {:.4f}, ({:.3f} sec/step)\n'.format(val_loss,time.time()-startval_time))

def test():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        #output = model(xtest.to(device))
        output = model(xtrain[:,:,:sampleSize].to(device).to(device))
        pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
        ans = mu_law_decode(pred)
        sf.write('./vsCorpus/result.wav', ans, sample_rate)
        print('stored done\n')
    
def train(epoch):
    model.train()
    idx = np.arange(xtrain.shape[-1] - 2*sampleSize)
    np.random.shuffle(idx)
    idx=np.array([0]*10000)
    for i, ind in enumerate(idx):
        start_time = time.time()
        data, target = xtrain[:,:,ind:ind+sampleSize].to(device), ytrain[:,ind:ind+sampleSize].to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f} , ({:.3f} sec/step)'.format(
                epoch, i, len(idx),100. * i / len(idx), loss.item(),time.time() - start_time))
        if i % 100 == 0:
            val()
            test()
            #torch.save(model, 'torchmodel')

In [ ]:
for epoch in range(1):
    train(epoch)
    test()

Train Epoch: 0 [0/10000 (0%)] Loss: 5.423798 , (1.764 sec/step)

val set:  5.4063, (0.701 sec/step)

stored done

Train Epoch: 0 [1/10000 (0%)] Loss: 5.300791 , (1.765 sec/step)
Train Epoch: 0 [2/10000 (0%)] Loss: 5.177410 , (1.764 sec/step)
Train Epoch: 0 [3/10000 (0%)] Loss: 5.073626 , (1.765 sec/step)
Train Epoch: 0 [4/10000 (0%)] Loss: 4.994006 , (1.768 sec/step)
Train Epoch: 0 [5/10000 (0%)] Loss: 4.922551 , (1.769 sec/step)
Train Epoch: 0 [6/10000 (0%)] Loss: 4.849060 , (1.770 sec/step)
Train Epoch: 0 [7/10000 (0%)] Loss: 4.772943 , (1.769 sec/step)
Train Epoch: 0 [8/10000 (0%)] Loss: 4.696674 , (1.770 sec/step)
Train Epoch: 0 [9/10000 (0%)] Loss: 4.623614 , (1.772 sec/step)
Train Epoch: 0 [10/10000 (0%)] Loss: 4.556101 , (1.771 sec/step)
Train Epoch: 0 [11/10000 (0%)] Loss: 4.494143 , (1.772 sec/step)
Train Epoch: 0 [12/10000 (0%)] Loss: 4.436659 , (1.773 sec/step)
Train Epoch: 0 [13/10000 (0%)] Loss: 4.382591 , (1.771 sec/step)
Train Epoch: 0 [14/10000 (0%)] Loss: 4.331529 , (1

Train Epoch: 0 [125/10000 (1%)] Loss: 2.652402 , (1.775 sec/step)
Train Epoch: 0 [126/10000 (1%)] Loss: 2.646629 , (1.778 sec/step)
Train Epoch: 0 [127/10000 (1%)] Loss: 2.641489 , (1.776 sec/step)
Train Epoch: 0 [128/10000 (1%)] Loss: 2.636522 , (1.776 sec/step)
Train Epoch: 0 [129/10000 (1%)] Loss: 2.630604 , (1.776 sec/step)
Train Epoch: 0 [130/10000 (1%)] Loss: 2.624315 , (1.776 sec/step)
Train Epoch: 0 [131/10000 (1%)] Loss: 2.619361 , (1.777 sec/step)
Train Epoch: 0 [132/10000 (1%)] Loss: 2.614160 , (1.777 sec/step)
Train Epoch: 0 [133/10000 (1%)] Loss: 2.608438 , (1.777 sec/step)
Train Epoch: 0 [134/10000 (1%)] Loss: 2.603619 , (1.777 sec/step)
Train Epoch: 0 [135/10000 (1%)] Loss: 2.598969 , (1.774 sec/step)
Train Epoch: 0 [136/10000 (1%)] Loss: 2.592786 , (1.776 sec/step)
Train Epoch: 0 [137/10000 (1%)] Loss: 2.587921 , (1.778 sec/step)
Train Epoch: 0 [138/10000 (1%)] Loss: 2.583060 , (1.776 sec/step)
Train Epoch: 0 [139/10000 (1%)] Loss: 2.577425 , (1.776 sec/step)
Train Epoc

Train Epoch: 0 [249/10000 (2%)] Loss: 2.178464 , (1.776 sec/step)
Train Epoch: 0 [250/10000 (2%)] Loss: 2.175571 , (1.776 sec/step)
Train Epoch: 0 [251/10000 (3%)] Loss: 2.172871 , (1.776 sec/step)
Train Epoch: 0 [252/10000 (3%)] Loss: 2.170161 , (1.776 sec/step)
Train Epoch: 0 [253/10000 (3%)] Loss: 2.167066 , (1.776 sec/step)
Train Epoch: 0 [254/10000 (3%)] Loss: 2.164191 , (1.776 sec/step)
Train Epoch: 0 [255/10000 (3%)] Loss: 2.161444 , (1.775 sec/step)
Train Epoch: 0 [256/10000 (3%)] Loss: 2.158766 , (1.776 sec/step)
Train Epoch: 0 [257/10000 (3%)] Loss: 2.156239 , (1.775 sec/step)
Train Epoch: 0 [258/10000 (3%)] Loss: 2.153746 , (1.775 sec/step)
Train Epoch: 0 [259/10000 (3%)] Loss: 2.151366 , (1.775 sec/step)
Train Epoch: 0 [260/10000 (3%)] Loss: 2.149465 , (1.774 sec/step)
Train Epoch: 0 [261/10000 (3%)] Loss: 2.146936 , (1.775 sec/step)
Train Epoch: 0 [262/10000 (3%)] Loss: 2.144232 , (1.775 sec/step)
Train Epoch: 0 [263/10000 (3%)] Loss: 2.141868 , (1.776 sec/step)
Train Epoc

Train Epoch: 0 [373/10000 (4%)] Loss: 1.918809 , (1.777 sec/step)
Train Epoch: 0 [374/10000 (4%)] Loss: 1.915112 , (1.775 sec/step)
Train Epoch: 0 [375/10000 (4%)] Loss: 1.911796 , (1.777 sec/step)
Train Epoch: 0 [376/10000 (4%)] Loss: 1.909569 , (1.777 sec/step)
Train Epoch: 0 [377/10000 (4%)] Loss: 1.908387 , (1.775 sec/step)
Train Epoch: 0 [378/10000 (4%)] Loss: 1.907368 , (1.775 sec/step)
Train Epoch: 0 [379/10000 (4%)] Loss: 1.905915 , (1.776 sec/step)
Train Epoch: 0 [380/10000 (4%)] Loss: 1.904416 , (1.777 sec/step)
Train Epoch: 0 [381/10000 (4%)] Loss: 1.903020 , (1.777 sec/step)
Train Epoch: 0 [382/10000 (4%)] Loss: 1.901557 , (1.777 sec/step)
Train Epoch: 0 [383/10000 (4%)] Loss: 1.899621 , (1.776 sec/step)
Train Epoch: 0 [384/10000 (4%)] Loss: 1.898305 , (1.776 sec/step)
Train Epoch: 0 [385/10000 (4%)] Loss: 1.896610 , (1.777 sec/step)
Train Epoch: 0 [386/10000 (4%)] Loss: 1.894896 , (1.777 sec/step)


In [ ]:
#model = torch.load('torchmodel')